## Keplerian Elements for Approximate Positions of the Major Planets
http://ssd.jpl.nasa.gov/txt/aprx_pos_planets.pdf

E.M. Standish  
Solar System Dynamics Group  
JPL/Caltec


general imports

In [1]:
import numpy as np
from numpy import sin, cos, tan, arctan, sqrt, pi, abs, radians
from numpy.linalg import norm
import matplotlib.pyplot as plt
from astropy.time import Time

In [2]:
%matplotlib inline

**Formulae for Computing Approximate Planetary Positions**  

Lower accuracy formulae for planetary positions have a number of important applications when one doesn't need the full accuracy of an integrated ephemeris. They are often used in observation scheduling, telescope pointing, and prediction of certain phenomena as well as in the planning and design of spacecraft missions.

Approximate positions of the major planets and Pluto may be found by using Keplerian formulae with their associated elements and rates. Such elements are not intended to represent any sort of mean; they are simply the result of being adjusted for a best fit. As such, it must be noted that the elements are not valid outside the given time-interval over which they were fit. 


|Keplerian elements       |                                                                 |
|:------------------------|:----------------------------------------------------------------|
|$a_o, \dot{a}$           |semi-major axis [au, au/cty]                                     |
|$e_o, \dot{e}$           |eccentricity [ , /cty]                                           |
|$I_o, \dot{I}$           |inclination [deg, deg/cty]                                       |
|$L_o, \dot{L}$           |mean longitude [deg, deg/cty]                                    |
|$\varpi_o, \dot{\varpi}$ |longitude of perihelion [deg, deg/cty] $\;\varpi = \omega+\Omega$|
|$\Omega, \dot{\Omega}$  |longitude of the ascending node [deg, deg/cty]                    |


**Table 1.**
Keplerian elements and their rates, with respect to the mean ecliptic  
and equinox of J2000, valid for the time-interval 1800 AD - 2050 AD.

---

In [3]:
#                          a            e            I            L             ϖ             Ω 
#                          au                       deg          deg           deg           deg
elements = np.array([( 0.38709927, 0.20563593,  7.00497902, 252.25032350,  77.45779628,  48.33076593),  # Mercury
                     ( 0.72333566, 0.00677672,  3.39467605, 181.97909950, 131.60246718,  76.67984255),  # Venus
                     ( 1.00000261, 0.01671123, -0.00001531, 100.46457166, 102.93768193,   0.0       ),  # Earth moon barycenter
                     ( 1.52371034, 0.09339410,  1.84969142,  -4.55343205, -23.94362959,  49.55953891),  # Mars
                     ( 5.20288700, 0.04838624,  1.30439695,  34.39644051,  14.72847983, 100.47390909),  # Jupiter
                     ( 9.53667594, 0.05386179,  2.48599187,  49.95424423,  92.59887831, 113.66242448),  # Saturn
                     (19.18916464, 0.04725744,  0.77263783, 313.23810451, 170.95427630,  74.01692503),  # Uranus
                     (30.06992276, 0.00859048,  1.77004347, -55.12002969,  44.96476227, 131.78422574),  # Neptune
                     (39.48211675, 0.24882730, 17.14001206, 238.92903833, 224.06891629, 110.30393684)], # Pluto
                     dtype=[('a', '<f8'), ('e', '<f8'), ('I', '<f8'), ('L', '<f8'), ('ϖ', '<f8'), ('Ω', '<f8')])

#                      au/cty        /cty       deg/cty       deg/cty         deg/cty      deg/cty
rates = np.array([( 0.00000037,  0.00001906, -0.00594749, 149472.67411175,  0.16047689, -0.12534081),  # Mercury
                  ( 0.00000390, -0.00004107, -0.00078890,  58517.81538729,  0.00268329, -0.27769418),  # Venus
                  ( 0.00000562, -0.00004392, -0.01294668,  35999.37244981,  0.32327364,  0.0       ),  # Earth moon barycenter
                  ( 0.00001847,  0.00007882, -0.00813131,  19140.30268499,  0.44441088, -0.29257343),  # Mars
                  (-0.00011607, -0.00013253, -0.00183714,   3034.74612775,  0.21252668,  0.20469106),  # Jupiter
                  (-0.00125060, -0.00050991,  0.00193609,   1222.49362201, -0.41897216, -0.28867794),  # Saturn
                  (-0.00196176, -0.00004397, -0.00242939,    428.48202785,  0.40805281,  0.04240589),  # Uranus
                  ( 0.00026291,  0.00005105,  0.00035372,    218.45945325, -0.32241464, -0.00508664),  # Neptune
                  (-0.00031596,  0.00005170,  0.00004818,    145.20780515, -0.04062942, -0.01183482)], # Pluto
                  dtype=[('da', '<f8'),('de', '<f8'),('dI', '<f8'),('dL', '<f8'),('dϖ', '<f8'),('dΩ', '<f8')])

**JPL Ephemeris Time $(\mathrm{T_{eph}})$** is a relativistic coordinate time used by JPL for high precision ephemeris and has been used for the ephemerides in the Astronomical Almanac starting in 1984.

**Terrestrial Time $(\mathrm{TT})$** is the "standard epoch" for modern astrometric reference data, designated J2000.0, is expressed as a TT instant: J2000.0 is 2000 January 1, 12h TT (JD 2451545.0 TT) at the geocenter. TT runs at the same rate as a time scale based on SI seconds on the surface of the Earth and is a constant offset from TAI. This offset of $32.184$ s is to preserve continuity with previous "dynamical" time scales, Terrestrial Dynamical Time (TDT) and Ephemeris Time (ET). It differs from Terrestrial Time only by small periodic terms with an amplitude not exceeding 2 milliseconds.

$\mathrm{T_{eph}} \approx \mathrm{TT}$.

$\mathrm{TT} = \mathrm{TAI} + 32.184\ \mathrm{s}$.

Therefore,

$\mathrm{TT} =  \mathrm{UTC} + \Delta\mathrm{AT} + 32.184\ \mathrm{s}$.

In [4]:
# time = Time('2019-03-14 0:00:00'); time.jd
time = Time.now();
TT = time.tt.jd; TT

2458552.3081859103

**Compute each of the planets six elements**

$a = a_o + \dot{a}\mathrm{T}$, etc.,

where  $\mathrm{T}$ is the number of centuries past J2000.0 (JD 2451545.0 TT).

$\mathrm{T} = (\mathrm{TT}-2451545.0)\ /\ 36525$

In [5]:
T = (TT-2451545.0)/36525; T

0.19184964232471843

In [6]:
a = elements['a'] + T*rates['da'] # semi-major axis
e = elements['e'] + T*rates['de'] # eccentricity
I = radians(elements['I'] + T*rates['dI']) # inclination in radians
L = radians(elements['L'] + T*rates['dL']) # mean longitude in radians
ϖ = radians(elements['ϖ'] + T*rates['dϖ']) # longitude of perihelion in radians
Ω = radians(elements['Ω'] + T*rates['dΩ']) # longitude of the ascending node in radians

**Compute the argument of perihelion**

$\omega = \varpi - \Omega$

In [7]:
ω = ϖ - Ω        # argument of perihelion

**Compute the mean anomoly**
$M = L - \varpi$

function to modulus the mean anomoly so that $-180^\circ \leq \theta \leq +180^\circ$

In [8]:
def mod(θ):
    while θ > pi: θ -= 2*pi  # in radians
    return θ

vmod = np.vectorize(mod)

In [9]:
M = vmod(L - ϖ)  # mean anomaly

**Kepler's Equation**

$M = E - e \cdot \sin(E)$

Function to solve Kepler's equation using the Newton-Raphson method.

In [10]:
def newton(M, e, eps=1e-15, maxitr=100):
    E = M + e*sin(M)
    itr = 0; ΔE = 1
    while np.abs(ΔE) > eps:
        ΔM = M - (E - e*sin(E))
        ΔE = ΔM/(1 - e*cos(E)) # error at the current iter
        E += ΔE
        itr += 1
    return E

vnewton = np.vectorize(newton)

**Compute the eccentric anomaly**

In [11]:
E = vnewton(M,e)

The **true anomaly** can be derived from the eccentric anomaly

$\nu = 2 \arctan{\sqrt{\frac{1 + e}{1 - e}}}\cdot\tan{\frac{E}{2}}$

In [12]:
ν = (2 * arctan(sqrt((1 + e) / (1 - e)) * tan(E / 2)))

**Compute the heliocentric coordinates $\mathrm{\mathbf{r}'}$**, in the planets orbital plane with x-axis aligned toward perihelion.

$x = a\cdot(\cos{E}-e)$

$y = a\cdot\sqrt{1-e^2}\cdot sin{E}$

$z = 0$

In [24]:
x = a*(cos(E)-e)
y = a*sqrt(1-e**2)*sin(E)
z = 0

**Compute the coordinates $\mathrm{\mathbf{r}_{ecl}}$**, in the J2000 ecliptic plane with x-axis aligned toward the equinox.

$ x_{ecl} = x\ (\cos{\omega}\cos{\Omega}-\sin{\omega}\sin{\Omega}\cos{I}) + y\ (-\sin{\omega}\cos{\Omega}-\cos{\omega}\sin{\Omega}\cos{I})$

$ y_{ecl} = x\ (\cos{\omega}\sin{\Omega}+\sin{\omega}\cos{\Omega}\cos{I}) + y\ (-\sin{\omega}\sin{\Omega}+\cos{\omega}\cos{\Omega}\cos{I})$

$ z_{ecl} = x\ (\sin{\omega}\sin{I}) + y\ (\cos{\omega}\sin{I})$


**Rotation Matrices**

\begin{aligned}
R_x(\theta) &= \left[\begin{matrix}1 & 0 & 0\\
                    0 & \cos{\left (\theta \right )} & - \sin{\left (\theta \right )}\\
                    0 & \sin{\left (\theta \right )} & \cos{\left (\theta \right )}
                    \end{matrix}\right]\\[12pt]
R_y(\theta) &= \left[\begin{matrix}\cos{\left (\theta \right )} & 0 & \sin{\left (\theta \right )}\\
                    0 & 1 & 0\\
                    -\sin{\left (\theta \right )} & 0 & \cos{\left (\theta \right )}
                    \end{matrix}\right]\\[12pt]
R_z(\theta) &= \left[\begin{matrix}\cos{\left (\theta \right )} & - \sin{\left (\theta \right )} & 0\\
                    \sin{\left (\theta \right )} & \cos{\left (\theta \right )} & 0\\
                    0 & 0 & 1
                    \end{matrix}\right]
\end{aligned}

---
\begin{equation}\left[\begin{matrix}\cos{\left (\Omega \right )} & - \sin{\left (\Omega \right )} & 0\\\sin{\left (\Omega \right )} & \cos{\left (\Omega \right )} & 0\\0 & 0 & 1\end{matrix}\right]\cdot\left[\begin{matrix}1 & 0 & 0\\0 & \cos{\left (I \right )} & - \sin{\left (I \right )}\\0 & \sin{\left (I \right )} & \cos{\left (I \right )}\end{matrix}\right]\cdot\left[\begin{matrix}\cos{\left (\omega \right )} & - \sin{\left (\omega \right )} & 0\\\sin{\left (\omega \right )} & \cos{\left (\omega \right )} & 0\\0 & 0 & 1\end{matrix}\right]\cdot\left[\begin{matrix}x\\y\\z\end{matrix}\right]=\\[18pt]
\end{equation}

\begin{equation}
\left[\begin{matrix}x \left(- \sin{\left (\Omega \right )} \sin{\left (\omega \right )} \cos{\left (I \right )} + \cos{\left (\Omega \right )} \cos{\left (\omega \right )}\right) + y \left(- \sin{\left (\Omega \right )} \cos{\left (I \right )} \cos{\left (\omega \right )} - \sin{\left (\omega \right )} \cos{\left (\Omega \right )}\right) + z \sin{\left (I \right )} \sin{\left (\Omega \right )}\\x \left(\sin{\left (\Omega \right )} \cos{\left (\omega \right )} + \sin{\left (\omega \right )} \cos{\left (I \right )} \cos{\left (\Omega \right )}\right) + y \left(- \sin{\left (\Omega \right )} \sin{\left (\omega \right )} + \cos{\left (I \right )} \cos{\left (\Omega \right )} \cos{\left (\omega \right )}\right) - z \sin{\left (I \right )} \cos{\left (\Omega \right )}\\x \sin{\left (I \right )} \sin{\left (\omega \right )} + y \sin{\left (I \right )} \cos{\left (\omega \right )} + z \cos{\left (I \right )}\end{matrix}\right]
\end{equation}

In [14]:
x_ecl, y_ecl, z_ecl = (( cos(ω)*cos(Ω) - sin(ω)*sin(Ω)*cos(I) )*x + ( -sin(ω)*cos(Ω) - cos(ω)*sin(Ω)*cos(I) )*y,
                       ( cos(ω)*sin(Ω) + sin(ω)*cos(Ω)*cos(I) )*x + ( -sin(ω)*sin(Ω) + cos(ω)*cos(Ω)*cos(I) )*y,
                       (            sin(ω)*sin(I)             )*x + (             cos(ω)*sin(I)             )*y)

**Compute the equatorial coordinates $\mathrm{\mathbf{r}_{eq}}$**, in the J2000 equatorial plane.

In [15]:
ε = radians(23.43928)

In [16]:
x_eq = x_ecl
y_eq = cos(ε)*y_ecl - sin(ε)*z_ecl
z_eq = sin(ε)*y_ecl + cos(ε)*z_ecl

In [35]:
P = np.matrix([x[0],y[0],0]); P.T

matrix([[0.1057941 ],
        [0.33255183],
        [0.        ]])

In [28]:
α = np.matrix([[cos(ω[0]), -sin(ω[0]), 0], [sin(ω[0]), cos(ω[0]), 0], [0, 0, 1]])
β = np.matrix([[1, 0, 0], [0, cos(I[0]), -sin(I[0])], [0, sin(I[0]), cos(I[0])]])
γ = np.matrix([[cos(Ω[0]), -sin(Ω[0]), 0], [sin(Ω[0]), cos(Ω[0]), 0], [0, 0, 1]])
δ = np.matrix([[1, 0, 0], [0, cos(ϵ), -sin(ϵ)], [0, sin(ϵ), cos(ϵ)]])

In [29]:
α

matrix([[ 0.87307645, -0.48758333,  0.        ],
        [ 0.48758333,  0.87307645,  0.        ],
        [ 0.        ,  0.        ,  1.        ]])

In [30]:
β

matrix([[ 1.        ,  0.        ,  0.        ],
        [ 0.        ,  0.99253799, -0.12193583],
        [ 0.        ,  0.12193583,  0.99253799]])

In [31]:
γ

matrix([[ 0.66514279, -0.74671619,  0.        ],
        [ 0.74671619,  0.66514279,  0.        ],
        [ 0.        ,  0.        ,  1.        ]])

In [32]:
δ

matrix([[ 1.        ,  0.        ,  0.        ],
        [ 0.        ,  0.91748214, -0.39777698],
        [ 0.        ,  0.39777698,  0.91748214]])

In [34]:
γ*β*α*P.T

matrix([[-0.29983084],
        [ 0.17362679],
        [ 0.0416931 ]])